In [4]:
pip install opencv-python face_recognition


Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import face_recognition
import os
from sys import exit
import sys
# Function to load images and encode faces
def load_images_from_folder(folder):
    known_faces = []
    known_names = []

    for person_folder in os.listdir(folder):
        person_path = os.path.join(folder, person_folder)

        # Skip non-directory items
        if not os.path.isdir(person_path):
            continue

        for filename in os.listdir(person_path):
            image_path = os.path.join(person_path, filename)

            # Skip non-image files
            try:
                img = face_recognition.load_image_file(image_path)
            except (IOError, OSError, Exception) as e:
                print(f"Skipping {filename} in {person_folder} due to an error: {str(e)}")
                continue

            # Check if the image contains a face
            if len(face_recognition.face_encodings(img)) == 0:
                print(f"Skipping {filename} in {person_folder} as no face was found.")
                continue

            face_encoding = face_recognition.face_encodings(img)[0]

            # Use the name of the subdirectory as the person's name
            name = person_folder

            known_faces.append(face_encoding)
            known_names.append(name)

    return known_faces, known_names


# Function to recognize faces using camera
def recognize_faces(camera_index=0):
    # Load known faces and names from the dataset folder
    dataset_folder = "dataset"
    known_faces, known_names = load_images_from_folder(dataset_folder)

    # Initialize camera
    cap = cv2.VideoCapture(camera_index)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Find all face locations and face encodings in the current frame
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        # Loop through each face found in the frame
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Check if the face matches any known faces

            matches = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.5)

            name = "Unknown"

            # If a match is found, use the name of the known face
            if True in matches:
                first_match_index = matches.index(True)
                name = known_names[first_match_index]

            # Draw rectangle around the face and display the name
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        # Display the resulting frame
        cv2.imshow('Face Recognition', frame)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to start face recognition using camera
recognize_faces()


Skipping .DS_Store in Utsav due to an error: cannot identify image file 'dataset/Utsav/.DS_Store'


TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.fhog_object_detector, image: numpy.ndarray, upsample_num_times: int = 0) -> _dlib_pybind11.rectangles

Invoked with: <_dlib_pybind11.fhog_object_detector object at 0x10623ac30>, None, 1